# "Ball and Stick" - Soma with Hodgkin & Huxley conductances and passive dendrite

 

## Step 1: Setup

In [ ]:
# Setup inline plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# For Google Colab, this line installs NEURON
#!pip install neuron quantities

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s, MOhm, mS, mm, Ohm

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")
#h.load_file("stdlib.hoc") # need geom_nseg

In [ ]:
# Import other modules we need
import numpy as np

## Step 2: Define the circuit


In [ ]:
# A standard soma
soma = h.Section(name="soma")
soma.L = soma.diam = 12.6157 #um

In [ ]:
# Add a dendrite
dend = h.Section(name="dend")
dend.L = 500
dend.diam = 1

In [ ]:
dend.connect(soma(1.0),0.0)

In [ ]:
for x in soma.wholetree():
    print(x)

In [ ]:
print(soma)

### Query NEURON for the expected units for soma.L & soma.diam

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance and resistivity "everywhere"

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
h.units("Ra")

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
for sec in soma.wholetree():
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100 * Ohm*cm # resistivity 

### Add the Hodgkin-Huxley conductances

In [ ]:
# This model includes the transient Na+, persistent K+ and the leak conductances
soma.insert("hh")

That's almost too easy!

### Parametize the leak conductance G = 1/R

In [ ]:
G = 1.5 * nS  # R = 1/G in our RC circuit

In [ ]:
v_rest = -65*mV

In [ ]:
(G/area).rescale(S/cm**2)

In [ ]:
# Assign the leak conductance everywhere in soma
for seg in soma:
    seg.hh.gl = (G/area).rescale(S/cm**2)  # Compute specific conductance, and rescale to units of 'S/cm2'
    seg.hh.el = -54.3

In [ ]:
# Insert passive conductance in dendrite
dend.insert("pas")
# Assign parameters everywhere in the dendrite
for seg in dend:
    seg.pas.g = 0.001  # Passive conductance in S/cm2
    seg.pas.e = v_rest  # Leak reversal potential mV

In [ ]:
dend.nseg

For extended sections, this idiom configures a sensible number of compartments per section (nseg) based on radii and $R_\lambda$<br>
See: https://nrn.readthedocs.io/en/latest/guide/using_the_d_lambda_rule.html

In [ ]:
for sec in [dend]:
    sec.nseg = int((sec.L/(0.1*h.lambda_f(100))+.9)/2)*2 + 1

In [ ]:
dend.nseg

In [ ]:
dend.nseg = 10

In [ ]:
for seg in dend:
    print(seg)

In [ ]:
type(seg)

In [ ]:
dend.wholetree()

In [ ]:
#dend.nseg = 1 (come back later to try this out)

### Add soma current pulse

In [ ]:
stim = h.IClamp(soma(0.5))

In [ ]:
stim.delay = 200 * ms
stim.dur = 1 * ms  
stim.amp = 0.05 * nA

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
soma_v = h.Vector().record(soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

In [ ]:
recording_locations = np.linspace(0,1.0,11)

In [ ]:
recording_locations

In [ ]:
dend_recs = [h.Vector().record(dend(x)._ref_v) for x in recording_locations]

In [ ]:
dend_labels = ["dend(%.1f).v" % f for f in recording_locations]

In [ ]:
dend_labels

In [ ]:
h.finitialize( float(v_rest) )
h.continuerun( float(1000 * ms) )

## Step 4: Plot the results

In [ ]:
fig = plt.figure(figsize = (10,10))
axes = fig.subplots(2, 1)
# assign which subplot for each trace
# The first goes to first subplot, the rest to the second
axes2 = [axes[0]] + len(dend_recs)*[axes[1]]
recs = [soma_v] + dend_recs
labels = ["soma(0.5).v"] + dend_labels
for i in range(len(labels)):
    axes2[i].plot(t, recs[i], lw=2, label=labels[i])
    axes2[i].legend()
    axes2[i].set_xlabel("t [ms]", size=16)
    axes2[i].set_ylabel("v [mV]", size=16)
    axes2[i].axis([190, 220, -65.5, -60])

In [ ]:
# That was "passive" stimulation.
# Now what about an AP?
stim.amp = 0.3 * nA

In [ ]:
h.finitialize( float(v_rest) )
h.continuerun( float(1000 * ms) )

In [ ]:
fig = plt.figure(figsize = (10,10))
axes = fig.subplots(2, 1)
# assign which subplot for each trace
# The first goes to first subplot, the rest to the second
axes2 = [axes[0]] + len(dend_recs)*[axes[1]]
recs = [soma_v] + dend_recs
labels = ["soma(0.5).v"] + dend_labels
for i in range(len(labels)):
    axes2[i].plot(t, recs[i], lw=2, label=labels[i])
    axes2[i].legend()
    axes2[i].set_xlabel("t [ms]", size=16)
    axes2[i].set_ylabel("v [mV]", size=16)
    axes2[i].axis([190, 220, -80, 50])

Because the dendrite is passive, the AP attenuates significantly along it.

## Try what would happen if dend.nseg = 1!
It makes a big difference! What's going on? Consider: Can the voltage attenuate if there is only 1 isopotential compartment?